<a href="https://colab.research.google.com/github/outofray/Ultrasonography/blob/main/resnet_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
input_shape = (400, 400, 3)
filename = "gdrive/MyDrive/morrison pouch real time feedback/demo 3.avi"
class_names = ["Ascites: Negative.     Quality: High.", "Ascites: Negative.     Quality: Low.", "Ascites: Positive.     Quality: High.", "Ascites: Positive.     Quality: Low."]
weight_file = "gdrive/MyDrive/morrison pouch real time feedback/morrison_4class_weights_ResNet50V2_96.hdf5"
data_augmentation = tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=input_shape)

In [ ]:
print(class_names)

['Ascites: Negative.     Quality: High.', 'Ascites: Negative.     Quality: Low.', 'Ascites: Positive.     Quality: High.', 'Ascites: Positive.     Quality: Low.']


In [ ]:
def ResNet50V2 (img_array):

    base_model = tf.keras.applications.resnet_v2.ResNet50V2(
        include_top=False,
        weights="imagenet", #pre-trained from imagenet
        input_shape=input_shape)

    base_model.trainable = True  #set weights to be trainable

    # Fine-tune from this layer onwards
    fine_tune_at = 96

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # add top layer
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    batchnorm = tf.keras.layers.BatchNormalization()
    prediction_layer = tf.keras.layers.Dense(4)

    # put model together
    inputs = tf.keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = batchnorm(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)

    base_learning_rate = 0.0001
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])

    model.load_weights(weight_file)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    return (score)

In [ ]:
# open video
cap = cv2.VideoCapture(filename)

res=(400,400) #resulotion
fourcc = cv2.VideoWriter_fourcc(*'XVID') #codec
out = cv2.VideoWriter('gdrive/MyDrive/morrison pouch real time feedback/inference.mp4', fourcc, 12.0, res)

# open frames in loop
while(cap.isOpened()):
  ret, frame = cap.read()

  # set and crop zone of interest
  x = 140
  y = 40
  w = 400
  h = 400
  frame_cropped = frame[y:y + h, x:x + w]

  # process frames
  img_array = tf.keras.preprocessing.image.img_to_array(frame_cropped)
  img_array = tf.expand_dims(img_array, 0)  # Create a batch

  # get model prediction score
  score = ResNet50V2(img_array)

  result = "{} ({:.2f}%)".format(class_names[np.argmax(score)], 100 * np.max(score))

  feedback = "<<Hold the probe>>"

  # feedback prediction on frame
  cv2.putText(frame_cropped, result, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)

  if np.argmax(score) in [0, 2]:
    cv2.putText(frame_cropped, feedback, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)

  cv2_imshow(frame_cropped)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

  out.write(frame_cropped)

cap.release()
cv2.destroyAllWindows()